# TP3: Problemas NP-Completos

#### Funciones auxiliares


In [10]:
# Recibe el path con el archivo a leer.
# Leemos y devolvemos dos arrays. 
# - El primero corresponde a un array de listas, donde cada lista posee los jugadores que piden los diferentes medios y periodistas.
# - El segundo array posee todos los jugadores que aparecen en el array anterior, siendo los jugadores que Scaloni puede elegir, su preselección.
# Este algoritmo, devuelve una preselección sin orden.
def obtener_preseleccion_y_pedidos_de_la_prensa(path):
    with open(path, 'r') as archivo:
        lineas = archivo.readlines()

    pedidos_de_la_prensa = []
    preseleccion = set()

    for linea in lineas:
        # Eliminar saltos de línea y dividir la línea en preseleccion
        preseleccion_linea = linea.strip().split(',')

        # Agregar la lista de preseleccion a 'pedidos_de_la_prensa'
        pedidos_de_la_prensa.append(preseleccion_linea)

        # Agregar cada nombre individual a 'preseleccion' si no existe previamente
        preseleccion.update(set(preseleccion_linea))

    return pedidos_de_la_prensa, list(preseleccion)



In [11]:
# Recibe el path con el archivo a leer.
# Leemos y devolvemos dos arrays. 
# - El primero corresponde a un array de listas, donde cada lista posee los jugadores que piden los diferentes medios y periodistas.
# - El segundo array posee todos los jugadores que aparecen en el array anterior, siendo los jugadores que Scaloni puede elegir, su preselección.
# Este algoritmo, devuelve una preselección en orden a medida que aparecen los jugadores en el archivo.
def leer_archivo(path):
    with open(path, 'r') as archivo:
        lineas = archivo.readlines()

    pedidos_de_la_prensa = []
    preseleccion = []

    for linea in lineas:
        # Eliminar saltos de línea y dividir la línea en preseleccion
        preseleccion_linea = linea.strip().split(',')

        # Agregar la lista de preseleccion a 'pedidos_de_la_prensa'
        pedidos_de_la_prensa.append(preseleccion_linea)

        # Agregar cada nombre individual a 'preseleccion' si no existe previamente
        for nombre in preseleccion_linea:
            if nombre not in preseleccion:
                preseleccion.append(nombre)

    return pedidos_de_la_prensa, preseleccion


In [12]:
#Paths de los archivos a utilizar
path_5 = 'sets de la catedra/5.txt'
path_7 = 'sets de la catedra/7.txt'
path_10_pocos = 'sets de la catedra/10_pocos.txt'
path_10_todos = 'sets de la catedra/10_todos.txt'
path_10_varios = 'sets de la catedra/10_varios.txt'
path_15 = 'sets de la catedra/15.txt'
path_20 = 'sets de la catedra/20.txt'
path_75 = 'sets de la catedra/75.txt'
path_100 = 'sets de la catedra/100.txt'
path_200 = 'sets de la catedra/200.txt'






In [16]:
#Me guardo los resultados esperados, dados por la cátedra
class Resultado:
    def __init__(self, cantidad_minima, jugadores):
        self.cantidad_minima = cantidad_minima
        self.jugadores = jugadores

def obtener_resultados_esperados(nombre_archivo):
    with open(nombre_archivo, 'r') as file:
        lineas = file.readlines()
        resultados = []

        for linea in lineas:
            if "Cantidad mínima:" in linea:
                cantidad_minima = int(linea.split(":")[1].split("(")[0].strip())
                jugadores_linea = lineas[lineas.index(linea)]
                jugadores = [jugador.strip() for jugador in jugadores_linea.split("(")[1].split(")")[0].split(",")]
                resultados.append(Resultado(cantidad_minima, jugadores))

        return resultados
    
# Uso del código
nombre_del_archivo = "sets de la catedra/Resultados Esperados.txt"  # Reemplaza con la ruta de tu archivo
resultados = leer_archivo(nombre_del_archivo)

# Acceder a los datos almacenados en las estructuras
for resultado in resultados:
    print("Cantidad mínima:", resultado.cantidad_minima)
    print("Jugadores:", ", ".join(resultado.jugadores))
    print()

AttributeError: 'list' object has no attribute 'cantidad_minima'